## Preparación de texto para Machine Learning

Los datos de texto requieren una preparación especial antes de poder ser usados en modelos predictivos.

El texto debe analizarse para eliminar **STOPWORDS**. Luego las palabras deben transformarse a valores numéricos para poder ser usado por lo modelos de predicción, este proceso se llama **`Feature Extraction`** o **`Vectorization`** (vectorizacion).

In [1]:
import numpy as np
import pandas as pd

import matplotlib # Para ver la versión
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn # Para ver la versión

# Escalado
from sklearn.preprocessing import MinMaxScaler

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Train, Test
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Validacion
from sklearn.model_selection import StratifiedKFold

In [2]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"matplotlib=={matplotlib.__version__}")
print(f"seaborn=={sns.__version__}")
print(f"scikit-learn=={sklearn.__version__}")

numpy==1.20.3
pandas==1.2.4
matplotlib==3.4.2
seaborn==0.11.1
scikit-learn==1.5.1


In [37]:
df = pd.read_csv("../Data/movie_data.csv")

df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [41]:
df = pd.read_csv("../Data/movie_data.csv", sep = ",", encoding='utf-8')

df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [4]:
df.shape

(50000, 2)

In [5]:
df["sentiment"].value_counts()

0    25000
1    25000
Name: sentiment, dtype: int64

In [6]:
print(df.loc[0, "review"])

In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70's, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich family 

### Bag-of-Words

Para poder trabajar con texto debemos transformar el texto en números, un modelo simple y efectivo es el modelo **Bag-of-Words**.

El modelo transforma el texto en una matriz, donde las palabras son columnas y las filas son las ocurrencias o las veces que aparece cada palabra en todo el texto.

En Sklearn tenemos la función **`CountVectorizer()`**.

_**Documentación:** https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html_

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# Inicializamos un objeto CountVecrtorizer()
count_vectorizer = CountVectorizer()

frases = np.array(["The sun is shining",
                   "The weather is sweet", 
                   "The sun is shining, the weather is sweet, and one and one is two"])

# Entrenamos el modelo y transformamos los datos.
bag = count_vectorizer.fit_transform(frases)

bag

# bag es una matriz sparse, por lo que Python decide mostrarla de la siguiente manera

<3x9 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [9]:
bag.toarray()

array([[0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1],
       [2, 3, 2, 1, 1, 1, 2, 1, 1]])

In [10]:
# .vocaboulary_ retorna cada palabra con su indice correspondiente en un diccionario.

print(count_vectorizer.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [11]:
vocabulario_ordenado = sorted(count_vectorizer.vocabulary_, key = lambda x : count_vectorizer.vocabulary_[x]) 
vocabulario_ordenado

['and', 'is', 'one', 'shining', 'sun', 'sweet', 'the', 'two', 'weather']

In [12]:
pd.DataFrame(data = bag.toarray(), columns = vocabulario_ordenado)

,and,is,one,shining,sun,sweet,the,two,weather
0,0,1,0,1,1,0,1,0,0
1,0,1,0,0,0,1,1,0,1
2,2,3,2,1,1,1,2,1,1


### Term Frequency-Inverse Document Frequency (TF-IDF)

La siguiente técnica se puede traducir como **frecuencia de término-frecuencia inversa de documento**, se utiliza para disminuir el peso de las palabras que aparecen muchas veces en multiples documentos, la ecuación es la siguiente: 

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

* El producto de la frecuencia de término $(tf(t,d))$ y la frecuencia inversa de documento $(idf(t,d))$. Donde ***t*** es el número de terminos y ***d*** el número de documento.

* ***idf(t,d)*** se calcula con la siguiente ecuación:

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

* Donde $n_d$ es el número total de documentos 
* ***df (d,t)*** es el número de documentos ***d*** que contienen el término ***t*** 
* El logaritmo se utiliza para evitar que las bajas frecuencias de documentos no adquieran demasiado peso

#### Ejemplo:
$$\text{idf}("is", d3) = log \frac{3}{1+3} = -0.287$$


$$\text{tf-idf}("is",d3)= 3 \times (-0.287) = -0.863$$

En Sklearn tenemos la función **`TfidfTransformer()`**.

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

# Inicializamos un objeto Tfidf
tfidf = TfidfTransformer()

# Cambio la precisión de python a 2 decimales
#np.set_printoptions(precision = 2)

# Entrenamos el Tfidf y transformamos la variable bag
bag_tfidf = tfidf.fit_transform(bag).toarray()

bag_tfidf

array([[0.        , 0.43370786, 0.        , 0.55847784, 0.55847784,
        0.        , 0.43370786, 0.        , 0.        ],
       [0.        , 0.43370786, 0.        , 0.        , 0.        ,
        0.55847784, 0.43370786, 0.        , 0.55847784],
       [0.50238645, 0.44507629, 0.50238645, 0.19103892, 0.19103892,
        0.19103892, 0.29671753, 0.25119322, 0.19103892]])

In [15]:
pd.DataFrame(data = bag.toarray(), columns = vocabulario_ordenado)

,and,is,one,shining,sun,sweet,the,two,weather
0,0,1,0,1,1,0,1,0,0
1,0,1,0,0,0,1,1,0,1
2,2,3,2,1,1,1,2,1,1


In [14]:
pd.DataFrame(data = bag_tfidf, columns = vocabulario_ordenado)

,and,is,one,shining,sun,sweet,the,two,weather
0,0.000000,0.433708,0.000000,0.558478,0.558478,0.000000,0.433708,0.000000,0.000000
1,0.000000,0.433708,0.000000,0.000000,0.000000,0.558478,0.433708,0.000000,0.558478
2,0.502386,0.445076,0.502386,0.191039,0.191039,0.191039,0.296718,0.251193,0.191039


**Vamos a aplicar ambas técnicas a `movie_data.csv`.**

In [22]:
df = df.iloc[18_000:-18_000, :]

df.shape

(14000, 2)

In [26]:
# Faltaría primero limpiar el texto de cosas raras.
# Faltaría stopwords quitar
# faltaría lematizar

In [27]:
df.sentiment.value_counts()

0    7016
1    6984
Name: sentiment, dtype: int64

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df["review"], # X
                                                    df["sentiment"].values, # y
                                                    test_size = 0.3,
                                                    random_state = 42,
                                                    stratify = df["sentiment"].values
                                                   )

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape},  y_test: {y_test.shape}")

X_train: (9800,), y_train: (9800,)
X_test: (4200,),  y_test: (4200,)


In [31]:
# Inicializamos un objeto CountVecrtorizer()
count_vectorizer = CountVectorizer(max_features = 3_000)

# Entrenamos el modelo y transformamos los datos.
bag = count_vectorizer.fit_transform(X_train)

bag

<9800x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 1068419 stored elements in Compressed Sparse Row format>

In [32]:
# Inicializamos un objeto Tfidf
tfidf = TfidfTransformer()

# Entrenamos el Tfidf y transformamos la variable bag
bag_tfidf = tfidf.fit_transform(bag)

bag_tfidf

<9800x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 1068419 stored elements in Compressed Sparse Row format>

In [33]:
X_train = bag_tfidf.toarray()

X_train.shape, y_train.shape

((9800, 3000), (9800,))

In [34]:
#del bag_tfidf, bag

In [35]:
%%time

# GaussianNB()
model = GaussianNB()

# Entrenamiento
model.fit(X_train, y_train)

CPU times: user 254 ms, sys: 207 ms, total: 461 ms
Wall time: 783 ms


GaussianNB()

In [36]:
# Predicciones
X_test = count_vectorizer.transform(X_test)
X_test = tfidf.transform(X_test)
X_test = X_test.toarray()

yhat = model.predict(X_test)

# Métricas
print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
print("Accuracy:"     , accuracy_score(y_test, yhat))
print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
print("ROC AUC:"      , roc_auc_score(y_test, yhat))

Jaccard Index: 0.6766091367063333
Accuracy: 0.8071428571428572
Precisión: 0.8072823775484113
Sensibilidad: 0.8071168203901382
F1-score: 0.8071109714054772
ROC AUC: 0.8071168203901382


In [ ]:
Accuracy: 0.8071428571428572

In [ ]:
vocabulario_ordenado = sorted(count_vectorizer.vocabulary_, key = lambda x : count_vectorizer.vocabulary_[x]) 
vocabulario_ordenado

In [ ]:
for i in vocabulario_ordenado:
    print(i)

In [ ]:
################################################################################################################################